# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096912


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:01<00:25,  1.11it/s]

Rendering models:  13%|█▎        | 4/30 [00:07<00:37,  1.42s/it]

Rendering models:  17%|█▋        | 5/30 [00:07<00:28,  1.13s/it]

Rendering models:  20%|██        | 6/30 [00:07<00:22,  1.08it/s]

...

Rendering models:  87%|████████▋ | 26/30 [00:12<00:01,  3.97it/s]

Rendering models:  90%|█████████ | 27/30 [00:13<00:00,  3.21it/s]

Rendering models:  97%|█████████▋| 29/30 [00:13<00:00,  3.91it/s]

Rendering models: 100%|██████████| 30/30 [00:13<00:00,  3.72it/s]

aliabdulhameed                          0.001763
DevionJ                                 0.000356
Kartoon1x                               0.000311
jnarayanbvg                             0.000214
not-logged-in-c6917d9b19f41e6c2995      0.002547
awright5                                0.001843
jaksonA                               256.535584
TSM_Ginga                               0.002045
AmeenDeen                               2.051352
CScampstudent                           0.002216
not-logged-in-6c3c85931d83a1164a0d      0.005950
not-logged-in-96548507daf63e9306f6      0.000358
sn346808                                0.025596
Finn.Ralston                            0.001275
not-logged-in-14d09f70263c1fe15cc3      0.000466
ElisabethB                              0.162707
Stargazer007001                         0.000472
puic                                    0.035056
not-logged-in-ca0e5c102eb9a9d65ed3      0.333665
umtuck1                                 0.000481
Mongin              

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())